In [1]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModel
import torch
from pytorch_tabular.models.tab_transformer import TabTransformerModel
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Step 1: Load Dataset
def load_data(file_path):
    """Loads the dataset and provides an initial overview."""
    data = pd.read_csv(file_path)
    print("Initial Dataset Info:\n", data.info())
    print("\nSample Data:\n", data.head())
    return data
data = load_data("C:\\Users\\sriha\\Music\\Case Comps\\NEST\\Data\\category_updated.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257577 entries, 0 to 257576
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  257577 non-null  int64  
 1   NCT Number                  257577 non-null  object 
 2   Study Title                 257577 non-null  object 
 3   Study Status                257577 non-null  object 
 4   Brief Summary               257577 non-null  object 
 5   Conditions                  257577 non-null  object 
 6   Primary Outcome Measures    247086 non-null  object 
 7   Secondary Outcome Measures  185779 non-null  object 
 8   Other Outcome Measures      18272 non-null   object 
 9   Sponsor                     257577 non-null  object 
 10  Collaborators               83679 non-null   object 
 11  Sex                         257317 non-null  object 
 12  Age                         257577 non-null  object 
 13  Phases        

In [3]:
data = data.head()
data

,Unnamed: 0,NCT Number,Study Title,Study Status,Brief Summary,Conditions,Primary Outcome Measures,Secondary Outcome Measures,Other Outcome Measures,Sponsor,...,Age,Phases,Enrollment,Funder Type,Study Type,Study Design,Start Month,Start Quarter,Condition Category,Conditions_Category
0,0,NCT00559130,Efficacy Study of CytoSorb Hemoperfusion Devic...,Completed,The hypothesis of this study is use of CytoSor...,Acute Respiratory Distress Syndrome|Acute Lung...,Relative IL-6 levels as a percent (%) of basel...,"Ventilator Free Days, Reduction cytokines TNF-...",NaN,"MedaSorb Technologies, Inc",...,"ADULT, OLDER_ADULT",NaN,100.0,INDUSTRY,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,11,4,Other Rare or Unclassified,Other Rare or Unclassified
1,1,NCT00937664,Safety and Tolerability Study of AZD7762 in Co...,Not_Completed,The primary purpose of this study is to find o...,Cancer|Solid Tumors|Advanced Solid Malignancies,Assessment of adverse events (based on CTCAE v...,Pharmacokinetic effect of AZD7762 when adminis...,NaN,AstraZeneca,...,"ADULT, OLDER_ADULT",PHASE1,24.0,INDUSTRY,INTERVENTIONAL,Allocation: NON_RANDOMIZED|Intervention Model:...,7,3,Oncology,Oncology
2,2,NCT00441597,Does Atorvastatin Reduce Ischemia-Reperfusion ...,Completed,To study the impact of 3 day exposure to atorv...,Ischemia Reperfusion Injury|Cardiovascular Dis...,Annexin A 5 targeting in the non dominant then...,"workload during ischemic exercise, workload du...",NaN,Radboud University Medical Center,...,ADULT,PHASE4,30.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CRO...,2,1,Other Rare or Unclassified,Other Rare or Unclassified
3,3,NCT03296228,Comparison of Dynamic Radiographs in Determini...,Completed,The purpose of this study is to identify the f...,Adolescent Idiopathic Scoliosis,Investigate the flexibility equivalence of dif...,Incorporate these findings into the Lenke Clas...,NaN,The University of Hong Kong,...,"CHILD, ADULT",NaN,134.0,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,-1,-1,Other Rare or Unclassified,Non-Oncology
4,4,NCT00421603,A Placebo-Controlled Study of Mixed Amphetamin...,Completed,"The proposed protocol is a double-blind, place...",Cocaine Dependence,Three Weeks of Continuous Cocaine Abstinence a...,NaN,NaN,New York State Psychiatric Institute,...,ADULT,PHASE2,81.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2,1,Other Rare or Unclassified,Non-Oncology


In [4]:
clinicaltokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
clinicalmodel = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [3]:
# Combine all text attributes into a single column 'Unstructured'
data["Unstructured"] = data[
    ["Brief Summary", "Study Title", "Primary Outcome Measures", "Secondary Outcome Measures"]
].astype(str).agg(" [SEP] ".join, axis=1)

# Drop the original text columns
data = data.drop(columns=["Brief Summary", "Study Title", "Primary Outcome Measures", "Secondary Outcome Measures"])

# Display updated DataFrame
print(data)

        Unnamed: 0   NCT Number   Study Status  \
0                0  NCT00559130      Completed   
1                1  NCT00937664  Not_Completed   
2                2  NCT00441597      Completed   
3                3  NCT03296228      Completed   
4                4  NCT00421603      Completed   
...            ...          ...            ...   
257572      257572  NCT02360800      Completed   
257573      257573  NCT02352506      Completed   
257574      257574  NCT04996381      Completed   
257575      257575  NCT00380640      Completed   
257576      257576  NCT01844336      Completed   

                                               Conditions  \
0       Acute Respiratory Distress Syndrome|Acute Lung...   
1         Cancer|Solid Tumors|Advanced Solid Malignancies   
2       Ischemia Reperfusion Injury|Cardiovascular Dis...   
3                         Adolescent Idiopathic Scoliosis   
4                                      Cocaine Dependence   
...                              

In [13]:
data['Unstructured']

0         The hypothesis of this study is use of CytoSor...
1         The primary purpose of this study is to find o...
2         To study the impact of 3 day exposure to atorv...
3         The purpose of this study is to identify the f...
4         The proposed protocol is a double-blind, place...
                                ...                        
257572    Bleeding after redo cardiac surgery is a commo...
257573    Acute kidney injury (AKI) is a common complica...
257574    The investigators will develop an artificial i...
257575    The purpose of this study is to assess the eff...
257576    The purpose of this study is to evaluate the p...
Name: Unstructured, Length: 257577, dtype: object

In [7]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Function to extract ClinicalBERT embeddings in GPU-friendly batches
def clinical_extract_text_embeddings(text_list, batch_size=512):
    """Extracts embeddings for text data using ClinicalBERT with batch processing on GPU."""
    embeddings = []
    
    dataloader = DataLoader(text_list, batch_size=batch_size, shuffle=False, pin_memory=True)

    for batch_texts in tqdm(dataloader, desc="Processing Batches", unit="batch"):
        try:
            # Tokenization and moving inputs to GPU
            inputs = clinicaltokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512)

            # Ensure all tensors are moved to GPU
            inputs = {key: val.to(device, non_blocking=True) for key, val in inputs.items()}  

            with torch.no_grad():  # No gradient tracking for inference
                outputs = clinicalmodel(**inputs)

            # Compute mean pooling of token embeddings (Sentence Representation)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).to(device)  # Ensure GPU tensor

            embeddings.append(batch_embeddings)

            # Free GPU memory after every batch
            del inputs, outputs
            torch.cuda.empty_cache()

        except torch.cuda.OutOfMemoryError:
            print("\n⚠️ CUDA OOM Error: Reducing batch size and retrying...\n")
            torch.cuda.empty_cache()
            return clinical_extract_text_embeddings(text_list, batch_size=max(batch_size // 2, 1))  # Reduce batch size & retry

    if not embeddings:
        print("⚠️ No embeddings extracted for this chunk! Skipping...")
        return None  # Return None if extraction failed

    return torch.cat([e.to(device) for e in embeddings], dim=0).cpu().numpy()  # Ensure all tensors are on GPU before conversion

# Split dataset into 100 chunks
num_chunks = 1
chunk_size = len(data) // num_chunks

for i in range(num_chunks):
    print(f"\n🚀 Processing chunk {i+1}/{num_chunks}...")

    # Select subset of data for this chunk
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i != num_chunks - 1 else len(data)  # Last chunk gets remaining data
    chunk = data.iloc[start_idx:end_idx].copy()  # Use .copy() to avoid warnings

    # Convert text column to list
    clinical_text_data = chunk["Unstructured"].astype(str).tolist()

    # Extract embeddings using batch processing on GPU
    embeddings_cpu = clinical_extract_text_embeddings(clinical_text_data, batch_size=512)  # Returns NumPy array

    # **Fix: Skip saving if embeddings is None (means extraction failed)**
    if embeddings_cpu is None:
        print(f"⚠️ Skipping chunk {i+1} due to failed embedding extraction.")
        continue  # Skip to the next chunk

    # Create DataFrame with embedding columns
    clinical_embedding_df = pd.DataFrame(embeddings_cpu, index=chunk.index)
    clinical_embedding_df.columns = [f"Unstructured_embed_{j}" for j in range(embeddings_cpu.shape[1])]

    # Merge embeddings with original text in the chunk
    chunk = pd.concat([chunk, clinical_embedding_df], axis=1)

    # Save the chunk with both text and embeddings
    output_filename = f"NEST_chunk_clinical_{i+1}.csv"
    chunk.to_csv(output_filename, index=True)
    
    print(f"✅ Saved chunk {i+1} to {output_filename}")

    # Free GPU memory after each chunk
    del embeddings_cpu, clinical_embedding_df, chunk
    torch.cuda.empty_cache()

print("\n🎉 All chunks successfully saved as separate CSV files!")


Using device: cuda

🚀 Processing chunk 1/1...


Processing Batches:   0%|          | 0/1 [00:00<?, ?batch/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [15]:
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader

# Load BioBERT model & tokenizer
biotokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
biomodel = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1").to("cuda")  # Move model to GPU

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Function to extract BioBERT embeddings in GPU-friendly batches
def bio_extract_text_embeddings(text_list, batch_size=512):
    """Extracts embeddings for text data using BioBERT with batch processing on GPU."""
    embeddings = []
    
    dataloader = DataLoader(text_list, batch_size=batch_size, shuffle=False)

    for batch_texts in tqdm(dataloader, desc="Processing Batches", unit="batch"):
        try:
            # Tokenization and moving inputs to GPU
            inputs = biotokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
            inputs = {key: val.to(device) for key, val in inputs.items()}  # Ensure all tensors are on GPU
            
            with torch.no_grad():  # No gradient tracking for inference
                outputs = biomodel(**inputs)

            # Compute mean pooling of token embeddings (Sentence Representation)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)  # Stays on GPU

            embeddings.append(batch_embeddings)

            # Free GPU memory after every batch
            del inputs, outputs
            torch.cuda.empty_cache()

        except torch.cuda.OutOfMemoryError:
            print("\n⚠️ CUDA OOM Error: Reducing batch size and retrying...\n")
            torch.cuda.empty_cache()
            return bio_extract_text_embeddings(text_list, batch_size=max(batch_size // 2, 1))  # Reduce batch size & retry

    return torch.cat(embeddings, dim=0).cpu().numpy()  # Move embeddings to CPU & convert to NumPy

# Split dataset into 100 chunks
num_chunks = 100
chunk_size = len(data) // num_chunks

for i in range(num_chunks):
    print(f"\n🚀 Processing chunk {i+1}/{num_chunks}...")

    # Select subset of data for this chunk
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i != num_chunks - 1 else len(data)  # Last chunk gets remaining data
    chunk = data.iloc[start_idx:end_idx].copy()  # Use .copy() to avoid warnings

    # Convert text column to list
    bio_text_data = chunk["Unstructured"].astype(str).tolist()

    # Extract embeddings using batch processing on GPU
    embeddings_cpu = bio_extract_text_embeddings(bio_text_data, batch_size=512)  # Returns NumPy array

    # Create DataFrame with embedding columns
    bioembedding_df = pd.DataFrame(embeddings_cpu, index=chunk.index)
    bioembedding_df.columns = [f"Unstructured_embed_{j}" for j in range(embeddings_cpu.shape[1])]

    # Merge embeddings with original text in the chunk
    chunk = pd.concat([chunk, bioembedding_df], axis=1)

    # Save the chunk with both text and embeddings
    output_filename = f"NEST_chunk_{i+1}.csv"
    chunk.to_csv(output_filename, index=True)
    
    print(f"✅ Saved chunk {i+1} to {output_filename}")

    # Free GPU memory after each chunk
    del embeddings_cpu, bioembedding_df, chunk
    torch.cuda.empty_cache()

print("\n🎉 All chunks successfully saved as separate CSV files!")


Using device: cuda

🚀 Processing chunk 1/100...


Processing Batches:  17%|█▋        | 1/6 [00:40<03:21, 40.29s/batch]


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling cublasLtMatmul with transpose_mat1 0 transpose_mat2 0 m 768 n 262144 k 768 mat1_ld 768 mat2_ld 768 result_ld 768 abcType 0 computeType 68 scaleType 0

In [5]:
import glob

# Load all chunk files and concatenate
files = glob.glob("NEST_chunk_clinical*.csv")
df_list = [pd.read_csv(f) for f in files]
final_df = pd.concat(df_list, axis=0)
final_df.head(10)

# Save final combined dataset
final_df.to_csv("NEST_clinical_embeddings_full.csv", index=False)
print("🎯 Merged all chunks into NEST_final_embeddings.csv")


🎯 Merged all chunks into NEST_final_embeddings.csv


In [4]:
# Load BioBERT model & tokenizer
clinicaltokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
clinicalmodel = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT").to("cuda")  # Move model to GPU

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Function to extract BioBERT embeddings in GPU-friendly batches
def clinical_extract_text_embeddings(text_list, batch_size=512):
    """Extracts embeddings for text data using BioBERT with batch processing on GPU."""
    embeddings = []
    
    dataloader = DataLoader(text_list, batch_size=batch_size, shuffle=False)

    for batch_texts in tqdm(dataloader, desc="Processing Batches", unit="batch"):
        try:
            # Tokenization and moving inputs to GPU
            inputs = clinicaltokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
            inputs = {key: val.to(device) for key, val in inputs.items()}  # Ensure all tensors are on GPU
            
            with torch.no_grad():  # No gradient tracking for inference
                outputs = clinicalmodel(**inputs)

            # Compute mean pooling of token embeddings (Sentence Representation)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)  # Stays on GPU

            embeddings.append(batch_embeddings)

            # Free GPU memory after every batch
            del inputs, outputs
            torch.cuda.empty_cache()

        except torch.cuda.OutOfMemoryError:
            print("\n⚠️ CUDA OOM Error: Reducing batch size and retrying...\n")
            torch.cuda.empty_cache()
            return clinical_extract_text_embeddings(text_list, batch_size=max(batch_size // 2, 1))  # Reduce batch size & retry

    return torch.cat(embeddings, dim=0).cpu().numpy()  # Move embeddings to CPU & convert to NumPy

# Split dataset into 100 chunks
num_chunks = 100
chunk_size = len(data) // num_chunks

for i in range(num_chunks):
    print(f"\n🚀 Processing chunk {i+1}/{num_chunks}...")

    # Select subset of data for this chunk
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i != num_chunks - 1 else len(data)  # Last chunk gets remaining data
    chunk = data.iloc[start_idx:end_idx].copy()  # Use .copy() to avoid warnings

    # Convert text column to list
    clinical_text_data = chunk["Unstructured"].astype(str).tolist()

    # Extract embeddings using batch processing on GPU
    embeddings_cpu = clinical_extract_text_embeddings(clinical_text_data, batch_size=512)  # Returns NumPy array

    # Create DataFrame with embedding columns
    clinicalembedding_df = pd.DataFrame(embeddings_cpu, index=chunk.index)
    clinicalembedding_df.columns = [f"Unstructured_embed_{j}" for j in range(embeddings_cpu.shape[1])]

    # Merge embeddings with original text in the chunk
    chunk = pd.concat([chunk, clinicalembedding_df], axis=1)

    # Save the chunk with both text and embeddings
    output_filename = f"NEST_chunk_clinical{i+1}.csv"
    chunk.to_csv(output_filename, index=True)
    
    print(f"✅ Saved chunk {i+1} to {output_filename}")

    # Free GPU memory after each chunk
    del embeddings_cpu, clinicalembedding_df, chunk
    torch.cuda.empty_cache()

print("\n🎉 All chunks successfully saved as separate CSV files!")


Using device: cuda

🚀 Processing chunk 1/100...


Processing Batches: 100%|██████████| 6/6 [02:22<00:00, 23.83s/batch]


✅ Saved chunk 1 to NEST_chunk_clinical1.csv

🚀 Processing chunk 2/100...


Processing Batches: 100%|██████████| 6/6 [02:23<00:00, 23.86s/batch]


✅ Saved chunk 2 to NEST_chunk_clinical2.csv

🚀 Processing chunk 3/100...


...

✅ Saved chunk 98 to NEST_chunk_clinical98.csv

🚀 Processing chunk 99/100...


Processing Batches: 100%|██████████| 6/6 [02:16<00:00, 22.69s/batch]


✅ Saved chunk 99 to NEST_chunk_clinical99.csv

🚀 Processing chunk 100/100...


Processing Batches: 100%|██████████| 6/6 [02:16<00:00, 22.82s/batch]


✅ Saved chunk 100 to NEST_chunk_clinical100.csv

🎉 All chunks successfully saved as separate CSV files!
